# Distillation

In [1]:
import torch

In [2]:
# Define the teacher and student models
teacher = torch.nn.Linear(10, 5)
student = torch.nn.Linear(10, 5)

# Define the loss function
loss_fn = torch.nn.KLDivLoss(reduction='batchmean')

# Define the optimizer and learning rate schedule
optimizer = torch.optim.SGD(student.parameters(), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
